<img style="float: left;padding: 1.3em" src="https://indico.in2p3.fr/event/18313/logo-786578160.png">  

This notebook puts together tutorials available at the [Gravitational-Wave Open Science Center (GWOSC) website](https://www.gw-openscience.org)

Topics:

* Plotting and manipulating publicly available gravitational-wave posterior samples. 
* Carrying out parameter estimation on open gravitational-wave data.

One of the modules we will be using in this notebook is [Bilby](https://lscsoft.docs.ligo.org/bilby/).  Bilby is a user-friendly Bayesian inference library primarily designed for inference of compact binary coalescence events in interferometric data.

# Part 3.1:  Parameter estimation for compact object mergers -- Using and interpreting posterior samples

This is a simple demonstration on loading and viewing public Bayesian inference results pertaining to gravitational-wave signals.

The data used here is downloaded from the public DCC page [LIGO-P1800370](https://dcc.ligo.org/LIGO-P1800370/public).

## Installation

To deal with time series we use [GWPy](https://gwpy.github.io)'s `TimeSeries`.

To generate waveforms to compare to the data, we need [LALSuite](https://lscsoft.docs.ligo.org/lalsuite/).

In [1]:
# -- Use the following line for google colab
! pip install -q 'corner==2.0.1' 'astropy==4.0.3'
! pip install -q 'lalsuite==6.82' 'bilby==1.0.4' 'gwpy==2.0.2' #2.0.2

     |████████████████████████████████| 10.2 MB 6.5 MB/s 
     |████████████████████████████████| 27.3 MB 5.2 MB/s 
     |████████████████████████████████| 11.6 MB 18.7 MB/s 
     |████████████████████████████████| 1.4 MB 52.5 MB/s 
     |████████████████████████████████| 1.6 MB 48.6 MB/s 
     |████████████████████████████████| 51 kB 8.1 MB/s 
     |████████████████████████████████| 87 kB 8.5 MB/s 
     |████████████████████████████████| 45 kB 4.2 MB/s 
     |████████████████████████████████| 55 kB 3.8 MB/s 
     |████████████████████████████████| 3.6 MB 27.3 MB/s 


**Important**: With Google Colab, you may need to restart the runtime after running the cell above.

## Initialization

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
import pandas as pd
import corner
import bilby

We check the Bilby version

In [3]:
print(bilby.__version__)

1.0.4: release


## Get the data

Select the GW150914 event.

In [4]:
label = 'GW150914'
filename = label+'_GWTC-1.hdf5'

# If you do not have wget installed, simply manually download 
# https://dcc.ligo.org/LIGO-P1800370/public/GW150914_GWTC-1.hdf5 
# from your browser
! wget https://dcc.ligo.org/LIGO-P1800370/public/{filename}

--2021-12-20 17:09:35--  https://dcc.ligo.org/LIGO-P1800370/public/GW150914_GWTC-1.hdf5
Resolving dcc.ligo.org (dcc.ligo.org)... 131.215.125.144
Connecting to dcc.ligo.org (dcc.ligo.org)|131.215.125.144|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://dcc.ligo.org/public/0157/P1800370/005/GW150914_GWTC-1.hdf5 [following]
--2021-12-20 17:09:35--  https://dcc.ligo.org/public/0157/P1800370/005/GW150914_GWTC-1.hdf5
Reusing existing connection to dcc.ligo.org:443.
HTTP request sent, awaiting response... 200 OK
Length: 7026464 (6.7M)
Saving to: ‘GW150914_GWTC-1.hdf5’

GW150914_GWTC-1.hdf 100%[===================>]   6.70M  --.-KB/s    in 0.1s    

2021-12-20 17:09:36 (57.0 MB/s) - ‘GW150914_GWTC-1.hdf5’ saved [7026464/7026464]



In [5]:
posterior = h5py.File('./'+filename, 'r')

### Looking into the file structure

[Hdf5](https://www.hdfgroup.org/solutions/hdf5/) (hierarchical data format) files work a bit like dictionaries.

In [ ]:
print('This file contains four datasets: ', posterior.keys())

This data file contains several datasets.
* Two datasets use separate models (approximants) for the gravitational waveform, namely `IMRPhenomPv2` and `SEOBNRv3`.  [See the [paper](https://dcc.ligo.org/LIGO-P1800307) for more details.]
* There is a joint dataset that combines equal numbers of samples from each individual model.
* Finally, there is a dataset containing samples drawn from the prior used for the analyses.

In [ ]:
print(posterior['Overall_posterior'].dtype.names)

Here are some brief descriptions of these parameters and their uses:

 * `luminosity_distance_Mpc`: luminosity distance [Mpc]

 * `m1_detector_frame_Msun`: primary (larger) black hole mass (detector frame) [solar mass]

 * `m2_detector_frame_Msun`: secondary (smaller) black hole mass (detector frame) [solar mass]

 * `right_ascension`, `declination`: right ascension and declination of the source [rad].

 * `costheta_jn`: cosine of the angle between line of sight and total angular momentum vector of the source.

 * `spin1`, `costilt1`: primary (larger) black hole spin magnitude (dimensionless) and cosine of the zenith angle between the spin and the orbital angular momentum vector of the source.

 * `spin2`, `costilt2`: secondary (smaller) black hole spin magnitude (dimensionless) and cosine of the zenith angle between the spin and the orbital angular momentum vector of the source.

A convenient (and pretty) way to load up an array of samples is to feed it as a NumPy array to [pandas](https://pandas.pydata.org/):

In [ ]:
samples = pd.DataFrame.from_records(np.array(posterior['Overall_posterior']))

In [ ]:
samples

These are all the samples stored in the `Overall` dataset. 

## Plotting

We can plot all of them with, for instance, the usual [corner](https://corner.readthedocs.io/en/latest/) package:

In [ ]:
corner.corner(samples,labels=['costhetajn',
                              'distance [Mpc]',
                              'ra',
                              'dec',
                              'mass1 [Msun]',
                              'mass2 [Msun]',
                              'spin1',
                              'spin2',
                              'costilt1',
                              'costilt2']);

We can manualy select one parameter and plot the marginalised distributions from the four distinct data sets.  We do this for the `luminosity distance`:

In [ ]:
for label in ['prior', 'IMRPhenomPv2_posterior', 'SEOBNRv3_posterior', 'Overall_posterior']:
    plt.hist(posterior[label]['luminosity_distance_Mpc'], bins = 100, label=label.replace('_posterior',''), alpha=0.8, density=True)

plt.xlabel(r'$D_L (Mpc)$')
plt.ylabel('Probability Density Function')
plt.legend()
plt.show()

### Computing new quantities

The masses given are the ones measured at the detector, i.e., in the *detector frame*. To determine the actual (*source frame*) masses of the source black holes, we need to correct for the cosmological redshift of the gravitational wave. This forces us to assume a cosmology.

And, of course, we can inform Python about cosmology 😀

In [ ]:
import astropy.units as u
from astropy.cosmology import Planck15, z_at_value

We compute the redshift value for all the samples (using only their distance value). See [astropy.cosmology](http://docs.astropy.org/en/stable/api/astropy.cosmology.z_at_value.html) for implementation details, in particular how to make the following more efficient:

In [ ]:
z = np.array([z_at_value(Planck15.luminosity_distance, dist * u.Mpc) for dist in samples['luminosity_distance_Mpc']])

We add new entries to `samples`.

In [ ]:
print(samples.keys())
print(samples.shape)

In [ ]:
samples['m1_source_frame_Msun'] = samples['m1_detector_frame_Msun']/(1.0+z)
samples['m2_source_frame_Msun'] = samples['m2_detector_frame_Msun']/(1.0+z)
samples['redshift'] = z

In [ ]:
print(samples.keys())
print(samples.shape)

And we can plot the marginalised probability density functions:

In [ ]:
corner.corner(samples[['m1_source_frame_Msun',
                       'm2_source_frame_Msun',
                       'redshift']],
              labels=['m1 (source)',
                      'm2 (source)',
                      'z']);

## Calculating credible intervals
Let's see how we can use the [Bilby package](https://lscsoft.docs.ligo.org/bilby/) to calcuate summary statistics for the posterior, e.g., the median and 90% credible interval.

The [chirp mass](https://en.wikipedia.org/wiki/Chirp_mass) is an important quantity in GW physics.  It is the main driver of the GW phase.  Its definition is

$$
\mathcal{M} = \frac{(m_1m_2)^{3/5}}{(m_1+m_2)^{1/5}}
$$

In [ ]:
# Calculate the detector frame chirp mass
mchirp = ((samples['m1_detector_frame_Msun'] * samples['m2_detector_frame_Msun'])**(0.6))/\
         (samples['m1_detector_frame_Msun'] + samples['m2_detector_frame_Msun'])**(0.2)

# Initialize a SampleSummary object to describe the chirp mass posterior samples
chirp_mass_samples_summary = bilby.core.utils.SamplesSummary(samples=mchirp, average='median')

# Output the desired information
print('Median chirp mass: {:.1f} Msun'.format(chirp_mass_samples_summary.median))
print('90% credible interval for the chirp mass: [{:.1f}, {:.1f}] Msun'.format(chirp_mass_samples_summary.lower_absolute_credible_interval,
                                                                        chirp_mass_samples_summary.upper_absolute_credible_interval))

# Part 3.2: Parameter estimation on GW150914 using open data

This example estimates the non-spinning parameters of the binary black hole system using commonly used prior distributions.
   
Find more examples at https://lscsoft.docs.ligo.org/bilby/examples.html

## Initialization

We begin by importing some commonly used functions

In [ ]:
from bilby.core.prior import Uniform
from bilby.gw.conversion import convert_to_lal_binary_black_hole_parameters, generate_all_bbh_parameters

from gwpy.timeseries import TimeSeries

### Set up empty interferometers

We will be using data from the Hanford (H1) and Livinston (L1) ground-based gravitational wave detectors. To start, we create two "empty" interferometers. These are empty in the sense that they do not have any strain data. But, they know about the orientation and location of their respective namesakes. It may also be interesting to note that they are initialised with the planned design sensitivity power spectral density of advanced LIGO - we will overwrite this later on, but it is often useful for simulations, as we saw in a previous notebook.

In [ ]:
H1 = bilby.gw.detector.get_empty_interferometer("H1")
L1 = bilby.gw.detector.get_empty_interferometer("L1")

## Getting the data: GW150914

Once more, we pick GW150914 to demonstrate things.

Our first task is to obtain the relevant data.  To do so, we need to know the trigger time: we already know how to do this with `gwosc`.  [Alternatively we can look it up manually on the [GWOSC page](https://www.gw-openscience.org/events/GW150914/).]

In [ ]:
# -- Uncomment following line if running in Google Colab
! pip install -q 'gwosc==0.5.4'

In [ ]:
from gwosc.datasets import event_gps
time_of_event = event_gps('GW150914')
print(time_of_event)

We use [GWPy](https://gwpy.github.io/) to download the open strain data.

To analyse GW150914, we will use a 4s period duration centered on the event itself. It is standard to choose the data such that it always includes a "post trigger duration" of 2s. That is, there are always 2s of data after the trigger time. We therefore define all times relative to the trigger time, duration and this post-trigger duration.

In [ ]:
# Definite times in relation to the trigger time (time_of_event), duration and post_trigger_duration
post_trigger_duration = 2
duration = 4
analysis_start = time_of_event + post_trigger_duration - duration
analysis_end = analysis_start + duration

# Fetch the open data
H1_analysis_data = TimeSeries.fetch_open_data(
    "H1", analysis_start, analysis_end, sample_rate=4096, cache=True)

L1_analysis_data = TimeSeries.fetch_open_data(
    "L1", analysis_start, analysis_end, sample_rate=4096, cache=True)

Here, `H1_analysis_data` and its L1 counterpart are GWPy `TimeSeries` objects. Remember that as such we can readily plot the data itself:

In [ ]:
H1_analysis_data.plot()
plt.show()

## Initialise the Bilby interferometers with the strain data

Now, we pass the downloaded strain data to our `H1` and `L1` Bilby interferometer objects. For other methods to set the strain data, see the various `set_strain_data*` methods.

In [ ]:
H1.set_strain_data_from_gwpy_timeseries(H1_analysis_data)
L1.set_strain_data_from_gwpy_timeseries(L1_analysis_data)

### Download the power spectral data

Parameter estimation relies on having a power spectral density (PSD) - an estimate of the coloured noise properties of the data. Here, we will create a PSD using off-source data. For a review of methods to estimate PSDs, see, e.g. [Chatziioannou et al. (2019)](https://ui.adsabs.harvard.edu/abs/2019PhRvD.100j4004C/abstract).

Again, we need to download this from the open strain data. We start by figuring out the amount of data needed. In this case: 32 times the analysis duration. We fetch the segment with this duration immediately preceding the analysis segment.

In [ ]:
psd_duration = duration * 32
psd_start_time = analysis_start - psd_duration
psd_end_time = psd_start_time + psd_duration

H1_psd_data = TimeSeries.fetch_open_data(
    "H1", psd_start_time, psd_end_time, sample_rate=4096, cache=True)

L1_psd_data = TimeSeries.fetch_open_data(
    "L1", psd_start_time, psd_end_time, sample_rate=4096, cache=True)

Having obtained the data to generate the PSD, we now use the standard [gwpy psd](https://gwpy.github.io/docs/stable/api/gwpy.timeseries.TimeSeries.html#gwpy.timeseries.TimeSeries.psd) method to calculate the PSD. Here, the `psd_alpha` variable is converting the `roll_off` applied to the strain data into the fractional value used by `gwpy`. This applies a window with an appropriate shape to the time-domain data.

In [ ]:
psd_alpha = 2 * H1.strain_data.roll_off / duration
H1_psd = H1_psd_data.psd(fftlength=duration, overlap=0, window=("tukey", psd_alpha), method="median")
L1_psd = L1_psd_data.psd(fftlength=duration, overlap=0, window=("tukey", psd_alpha), method="median")

### Initialise the PSD
Now that we have PSDs for H1 and L1, we can overwrite the `power_spectal_density` attribute of our interferometers with a new PSD.

In [ ]:
H1.power_spectral_density = bilby.gw.detector.PowerSpectralDensity(
    frequency_array=H1_psd.frequencies.value, psd_array=H1_psd.value)
L1.power_spectral_density = bilby.gw.detector.PowerSpectralDensity(
    frequency_array=L1_psd.frequencies.value, psd_array=L1_psd.value)

### Looking at the data
Okay, we have spent a bit of time now downloading and initializing things. Let's check that everything makes sense. To do this, we'll plot our analysis data alongside the amplitude spectral density (ASD); this is just the square root of the PSD and has the right units to be comparable to the frequency-domain strain data.

In [ ]:
fig, ax = plt.subplots()
idxs = H1.strain_data.frequency_mask  # This is a boolean mask of the frequencies which we will use in the analysis
ax.loglog(H1.strain_data.frequency_array[idxs],
          np.abs(H1.strain_data.frequency_domain_strain[idxs]))
ax.loglog(H1.power_spectral_density.frequency_array[idxs],
          H1.power_spectral_density.asd_array[idxs])
ax.set_xlabel("Frequency [Hz]")
ax.set_ylabel("Strain [strain/$\sqrt{Hz}$]")
plt.show()

In [ ]:
fig, ax = plt.subplots()
idxs = L1.strain_data.frequency_mask  # This is a boolean mask of the frequencies which we will use in the analysis
ax.loglog(L1.strain_data.frequency_array[idxs],
          np.abs(L1.strain_data.frequency_domain_strain[idxs]))
ax.loglog(L1.power_spectral_density.frequency_array[idxs],
          L1.power_spectral_density.asd_array[idxs])
ax.set_xlabel("Frequency [Hz]")
ax.set_ylabel("Strain [strain/$\sqrt{Hz}$]")
plt.show()

What is happening at high frequencies? This is an artifact of the downsampling applied to the data.  Note that we downloaded the 4096Hz data which is downsampled from 16384Hz. We are not really interested in the data at these high frequencies so let's adjust the maximum frequency used in the analysis to 1024 Hz and plot things again.

In [ ]:
H1.maximum_frequency = 1024
L1.maximum_frequency = 1024

In [ ]:
fig, ax = plt.subplots()
idxs = H1.strain_data.frequency_mask
ax.loglog(H1.strain_data.frequency_array[idxs],
          np.abs(H1.strain_data.frequency_domain_strain[idxs]))
ax.loglog(H1.power_spectral_density.frequency_array[idxs],
          H1.power_spectral_density.asd_array[idxs])
ax.set_xlabel("Frequency [Hz]")
ax.set_ylabel("Strain [strain/$\sqrt{Hz}$]")
plt.show()

Okay, that is better. We will not analyse any data near to the artifact produced by downsampling.

Now we have some sensible data to analyse so let's get right on with doing the analysis!

## Low dimensional analysis

In general a compact binary coalescence signal is described by 15 parameters describing the masses, spins, orientation, and position of the two compact objects along with a time at which the signal merges. The goal of parameter estimation is to figure out what the data (and any cogent prior information) can tell us about the likely values of these parameters. This is called the *posterior distribution* of the parameters.

To start with, we analyse the data fixing all but a few of the parameters to known values (in other words the priors of these parameters are delta functions), this will enable us to run things in a few minutes rather than the many hours needed to do full parameter estimation.

We start by thinking about the mass of the system. We call the heavier black hole the primary and label its mass $m_1$ and that of the secondary (lighter) black hole $m_2$. In this way, we always define $m_1 \ge m_2$. It turns out that inferences about $m_1$ and $m_2$ are highly correlated, we will see exactly what this means later on.

Bayesian inference methods are powerful at figuring out highly correlated posteriors. But, we can ease the process by sampling in parameters which are not highly correlated. In particular, we define two new mass parameter to be the [chirp mass](https://en.wikipedia.org/wiki/Chirp_mass)

$$ \mathcal{M} = \frac{(m_1 m_2)^{3/5}}{(m_1 + m_2)^{1/5}} $$

and the mass ratio

$$ q = \frac{m_{2}}{m_1}\,. $$

If we sample (make inferences about) $\mathcal{M}$ and $q$, our code is much faster than if we use $m_1$ and $m_2$ directly! Note that so long as the equivalent prior is given, one can also sample in the component masses themselves and you will get the same answer, it is just much slower!

Once we have inferred $\mathcal{M}$ and $q$, we can then derive $m_1$ and $m_2$ from the resulting samples, as we will see below.

Let's run a short (~1min on a single 2.8GHz core), low-dimensional parameter estimation analysis. This is done by defining a prior dictionary where all parameters are fixed, except those that we want to vary.

### Create a prior

Here, we create a prior fixing everything except the chirp mass, mass ratio, phase and geocent_time parameters to fixed values. The first two were described above. The second two give the phase of the system and the time at which it merges.

In [ ]:
prior = bilby.core.prior.PriorDict()
prior['chirp_mass'] = Uniform(name='chirp_mass', minimum=30.0,maximum=32.5)
prior['mass_ratio'] = Uniform(name='mass_ratio', minimum=0.5, maximum=1)
prior['phase'] = Uniform(name="phase", minimum=0, maximum=2*np.pi)
prior['geocent_time'] = Uniform(name="geocent_time", minimum=time_of_event-0.1, maximum=time_of_event+0.1)
prior['a_1'] =  0.0
prior['a_2'] =  0.0
prior['tilt_1'] =  0.0
prior['tilt_2'] =  0.0
prior['phi_12'] =  0.0
prior['phi_jl'] =  0.0
prior['dec'] =  -1.2232
prior['ra'] =  2.19432
prior['theta_jn'] =  1.89694
prior['psi'] =  0.532268
prior['luminosity_distance'] = 412.066

## Create a likelihood

For Bayesian inference, we need to evaluate the likelihood. In Bilby, we create a **likelihood object**. This is the communication interface between the sampling part of Bilby and the data. Explicitly, when Bilby is sampling it only uses the `parameters` and `log_likelihood()` of the likelihood object. This means the likelihood can be arbitrarily complicated and the sampling part of Bilby will not mind a bit!

Let's create a `GravitationalWaveTransient`, a special inbuilt method carefully designed to wrap up evaluating the likelihood of a waveform model in some data.

In [ ]:
# First, put our "data" created above into a list of intererometers
# (the order is arbitrary)
interferometers = [H1, L1]

# Next create a dictionary of arguments to pass to LALSimulation (the part of
# LALSuite that calculates waveforms) via the Bilby waveform generator interface.
# This is where we specify the waveform approximant.
waveform_arguments = dict(
    waveform_approximant='IMRPhenomPv2', reference_frequency=100., catch_waveform_errors=True)

# Next, create a waveform_generator object. This wraps up some of the jobs of
# converting between parameters, etc.
waveform_generator = bilby.gw.WaveformGenerator(
    frequency_domain_source_model=bilby.gw.source.lal_binary_black_hole,
    waveform_arguments=waveform_arguments,
    parameter_conversion=convert_to_lal_binary_black_hole_parameters)

# Finally, create the likelihood, passing in what is needed to get going
likelihood = bilby.gw.likelihood.GravitationalWaveTransient(
    interferometers,
    waveform_generator,
    priors=prior,
    time_marginalization=True,
    phase_marginalization=True,
    distance_marginalization=False)

Note that we also specify `time_marginalization=True` and `phase_marginalization=True`. This is a trick often used in Bayesian inference. We analytically marginalize (integrate) over the time/phase of the system while sampling, effectively reducing the parameter space and making it easier to sample. Bilby will then figure out (after the sampling) posteriors for these marginalized parameters. For an introduction to this topic, see [Thrane & Talbot (2019)](https://arxiv.org/abs/1809.02293).

### Run the analysis

Now that the prior is set-up and the likelihood is set-up (with the data and the signal mode), we can run the sampler to get the posterior result. This function takes the likelihood and prior along with some options for how to do the sampling and how to save the data.

In [ ]:
result_short = bilby.run_sampler(
    likelihood, prior, sampler='dynesty', outdir='short', label="GW150914",
    conversion_function=bilby.gw.conversion.generate_all_bbh_parameters,
    sample="unif", nlive=500, dlogz=3  # <- Arguments are used to make things fast - not recommended for general use
)

### Looking at the outputs

The `run_sampler` returned `result_short`, a Bilby result object. The posterior samples are stored in a [pandas data frame](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html) (think of this like a spreadsheet); let's take a look at it.

In [ ]:
result_short.posterior

We can pull out specific parameters that we are interested in

In [ ]:
result_short.posterior["chirp_mass"]

This returned another `pandas` object. If you just want to get the numbers as a numpy array run

In [ ]:
Mc = result_short.posterior["chirp_mass"].values

We can then get some useful quantities such as the 90\% credible interval

In [ ]:
lower_bound = np.quantile(Mc, 0.05)
upper_bound = np.quantile(Mc, 0.95)
median = np.quantile(Mc, 0.5)
print("Mc = {:.1f} with a 90% C.I = {:.1f} -> {:.1f}".format(median, lower_bound, upper_bound))

We can then plot the chirp mass in a histogram adding a region to indicate the 90\% C.I.

In [ ]:
fig, ax = plt.subplots()
ax.hist(result_short.posterior["chirp_mass"], bins=20)
ax.axvspan(lower_bound, upper_bound, color='C1', alpha=0.4)
ax.axvline(median, color='C1')
ax.set_xlabel("chirp mass")
plt.show()

The result object also has in-built methods to make nice plots such as corner plots. You can add the priors if you are only plotting parameters which you sampled in, e.g.

In [ ]:
result_short.plot_corner(parameters=["chirp_mass", "mass_ratio", "geocent_time", "phase"], prior=True)

You can also plot lines indicating specific points. Here, we add the values recorded on [GWOSC](https://www.gw-openscience.org/events/GW150914/). Notably, these fall outside the bulk of the posterior uncertainty here. This is because we limited our prior. If instead we were to run the full analysis, these would agree nicely.

In [ ]:
parameters = dict(mass_1=36.2, mass_2=29.1)
result_short.plot_corner(parameters)

In this plot we start to see the correlation between $m_1$ and $m_2$ that we disucssed earlier.

### Meta data
The result object also stores meta data, such as the priors

In [ ]:
result_short.priors

and details of the analysis itself:

In [ ]:
result_short.sampler_kwargs["nlive"]

Finally, we can also extract the **Bayes factor** for the signal vs. Gaussian noise. This quantifies the probability that the analyzed segment constains a binary black hole signal compared to just containing noise.

In [ ]:
print("ln Bayes factor = {:.2f} +/- {:.2f}".format(
    result_short.log_bayes_factor, result_short.log_evidence_err))

### Other attributes of the result

In [ ]:
print(result_short.log_evidence)
print(result_short.log_evidence_err)
print(result_short.log_noise_evidence)
print(result_short.log_bayes_factor)
print(result_short.log_evidence - result_short.log_noise_evidence)

In [ ]:
plt.plot(result_short.log_likelihood_evaluations)

In [ ]:
_, _, histo = plt.hist(result_short.log_likelihood_evaluations, bins=100)

In [ ]:
print(result_short.covariance_matrix)

#print(result_short.nested_samples)
#print(result_short.samples)

In [ ]:
qty = 'chirp_mass'
#quantiles = (0.16, 0.84)
quantiles = (0.05, 0.95)
result_short.plot_single_density(qty, save=False, prior=result_short.priors[qty], quantiles=quantiles)
plt.yscale('log')
result_short.plot_single_density(qty, save=False, quantiles=quantiles, cumulative=True)
plt.yscale('linear')

In [ ]:
from IPython.display import Markdown as md
md(result_short.get_one_dimensional_median_and_error_bar(qty, quantiles=quantiles).string)

In [ ]:
result_short.posterior_volume